## Imports

In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)  # Muda o diretório para o nível anterior (a raiz do projeto)
print(os.getcwd())  # Verifique se agora está na raiz

/Users/patricia/Documents/code/python-code/behavior-detection/src


In [2]:
from sklearn import datasets
from time import time
import pandas as pd

## Data

In [3]:
digits = datasets.load_digits()

n_samples = len(digits.images)
X = digits.images
y = digits.target

# Flatten the images
X = X.reshape((n_samples, -1))
X = pd.DataFrame(X)

In [4]:
print(n_samples)
print(type(X))
print(X.head(5))
print(y[:5])

1797
<class 'pandas.core.frame.DataFrame'>
    0    1    2     3     4     5    6    7    8    9   ...   54   55   56  \
0  0.0  0.0  5.0  13.0   9.0   1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1  0.0  0.0  0.0  12.0  13.0   5.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2  0.0  0.0  0.0   4.0  15.0  12.0  0.0  0.0  0.0  0.0  ...  5.0  0.0  0.0   
3  0.0  0.0  7.0  15.0  13.0   1.0  0.0  0.0  0.0  8.0  ...  9.0  0.0  0.0   
4  0.0  0.0  0.0   1.0  11.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

    57   58    59    60    61   62   63  
0  0.0  6.0  13.0  10.0   0.0  0.0  0.0  
1  0.0  0.0  11.0  16.0  10.0  0.0  0.0  
2  0.0  0.0   3.0  11.0  16.0  9.0  0.0  
3  0.0  7.0  13.0  13.0   9.0  0.0  0.0  
4  0.0  0.0   2.0  16.0   4.0  0.0  0.0  

[5 rows x 64 columns]
[0 1 2 3 4]


In [5]:
from sklearn.model_selection import train_test_split

# Dividir X e y em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Preprocessing

In [6]:
import pandas as pd
from core.preprocessors.data_encoder import DataEncoder

# Codificar y_train
y_train = DataEncoder.encode_y(y_train)
# Codificar y_test
y_test = DataEncoder.encode_y(y_test)

# Pré-processar X_train
X_encoder = DataEncoder(num_classes=0, select_numerical=True)
X_encoder.fit(X_train)

X_train = X_encoder.transform(X_train)

# Pré-processar X_test usando o mesmo preprocessor
X_test = X_encoder.transform(X_test)

In [7]:
print(X_train.head(15))

      num_standard__x0  num_standard__x1  num_standard__x2  num_standard__x3  \
1734               0.0         -0.341698         -0.463360          0.508365   
855                0.0         -0.341698          0.784716         -0.658600   
1642               0.0         -0.341698         -1.087399         -0.425207   
175                0.0          0.759243          0.992729          0.975151   
925                0.0         -0.341698          0.160678          0.508365   
548                0.0         -0.341698          1.616768          0.975151   
1615               0.0         -0.341698         -1.087399         -0.658600   
334                0.0         -0.341698         -0.463360         -0.191814   
756                0.0         -0.341698         -1.087399         -0.658600   
433                0.0         -0.341698         -1.087399         -1.592172   
889                0.0          5.163004          2.240806          0.041579   
1675               0.0          0.759243

## Classifier

In [8]:
# Definir quais modelos e seletores utilizar
selected_models = [ 
    # 'Logistic Regression',
    # 'Decision Tree',
    'Random Forest',
    # 'Gradient Boosting',
    # 'SVM',
    # 'KNN',
    'XGBoost'
]
selected_selectors = ['none']

cv = 5  # Number of folds in the cross-validation
n_iter = 100
n_jobs = 4  # Number of processors to be used in the execution: -1 to use all processors

# Choose a scoring metric
scoring_metric = 'balanced_accuracy'  # Possible values: 'f1_macro', 'balanced_accuracy', 'roc_auc_ovr', etc.

## Usando BayesSearchCV (Otimização Bayesiana)

In [9]:
from core.training.skopt_bayesian_optimization_training import SkoptBayesianOptimizationTraining

training = SkoptBayesianOptimizationTraining()

#### Executar o treinamento
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

#### Exemplo de acesso aos modelos treinados
for model_key, model_info in trained_models.items():
    print(f"Modelo: {model_key}")
    print(f"Melhores Hiperparâmetros: {model_info['hyperparameters']}")
    print(f"Resultado CV: {model_info['cv_result']}\n") 

INFO:root:Training and evaluating Random Forest with Bayesian Optimization and none
INFO:root:Search space for Random Forest with selector none: {'classifier__n_estimators': [50, 100, 200], 'classifier__max_depth': [None, 3, 5, 10, 20, 30], 'classifier__min_samples_split': [2, 5, 10], 'classifier__min_samples_leaf': [1, 2, 4, 6, 8, 10]}


Training and evaluating Random Forest with Bayesian Optimization and none
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.928 total time=   0.1s
[CV 2/5] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.915 total time=   0.1s
[CV 3/5] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.922 total time=   0.2s


INFO:root:Iteration 1: tested parameters: [5, 8, 10, 50], score: 0.9264236629764995


[CV 4/5] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.950 total time=   0.2s
[CV 5/5] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.917 total time=   0.1s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 2/5] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=2, classifier__n_estimators=200;, score=0.947 total time=   0.6s
[CV 1/5] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=2, classifier__n_estimators=200;, score=0.947 total time=   0.6s
[CV 3/5] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=2, classifier__n_estimators=200;, score=0.936 total time=   0.6s
[CV 4/5] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=2, class

INFO:root:Iteration 2: tested parameters: [30, 10, 2, 200], score: 0.950972950544697


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 3/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=0.6, classifier__reg_lambda=0.6, classifier__subsample=0.7;, score=0.943 total time=   0.6s
[CV 2/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=0.6, classifier__reg_lambda=0.6, classifier__subsample=0.7;, score=0.962 total time=   0.6s
[CV 1/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=0.6, classifier__reg_lambda=0.6, classifier__subsample=0.7;, score=0.972 total time=   0.6s
[CV 4/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.05, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=0.6, classifier__reg_lambda=0.6, classifi

INFO:root:Iteration 99: tested parameters: [0.7, 0.05, 3, 150, 0.6, 0.6, 0.7], score: 0.9543289310285973


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.15, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=1.0, classifier__reg_lambda=0.4, classifier__subsample=0.95;, score=0.972 total time=   0.4s
[CV 3/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.15, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=1.0, classifier__reg_lambda=0.4, classifier__subsample=0.95;, score=0.946 total time=   0.4s
[CV 2/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.15, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=1.0, classifier__reg_lambda=0.4, classifier__subsample=0.95;, score=0.958 total time=   0.4s
[CV 4/5] END classifier__colsample_bytree=0.7, classifier__learning_rate=0.15, classifier__max_depth=3, classifier__n_estimators=150, classifier__reg_alpha=1.0, classifier__reg_lambda=0.4, class

INFO:root:Iteration 100: tested parameters: [0.7, 0.15, 3, 150, 1.0, 0.4, 0.95], score: 0.954335864070013
2024-10-22 17:43:11,869 | skopt_training | INFO | Best parameters: OrderedDict([('classifier__colsample_bytree', 0.7), ('classifier__learning_rate', 0.15), ('classifier__max_depth', 7), ('classifier__n_estimators', 300), ('classifier__reg_alpha', 1.0), ('classifier__reg_lambda', 0.2), ('classifier__subsample', 0.7)])
INFO:skopt_training:Best parameters: OrderedDict([('classifier__colsample_bytree', 0.7), ('classifier__learning_rate', 0.15), ('classifier__max_depth', 7), ('classifier__n_estimators', 300), ('classifier__reg_alpha', 1.0), ('classifier__reg_lambda', 0.2), ('classifier__subsample', 0.7)])
2024-10-22 17:43:11,890 | skopt_training | INFO | Best cross-validation score: 0.9663679749986758
INFO:skopt_training:Best cross-validation score: 0.9663679749986758
2024-10-22 17:43:11,910 | skopt_training | INFO | All hyperparameter combinations and their cross-validation results:
IN

Bayesian Optimization Best Result for XGBoost with none: 0.9663679749986758
Modelo: Random Forest_none
Melhores Hiperparâmetros: {'memory': None, 'steps': [('classifier', RandomForestClassifier(n_estimators=200))], 'verbose': False, 'classifier': RandomForestClassifier(n_estimators=200), 'classifier__bootstrap': True, 'classifier__ccp_alpha': 0.0, 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': None, 'classifier__max_samples': None, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__min_weight_fraction_leaf': 0.0, 'classifier__n_estimators': 200, 'classifier__n_jobs': None, 'classifier__oob_score': False, 'classifier__random_state': None, 'classifier__verbose': 0, 'classifier__warm_start': False}
Resultado CV: 0.9740151432220397

Modelo: XGBoost_none
Melhores Hiperparâmetros: {'memory': None, 'step

### Avaliação e logging

In [10]:
import importlib
import notebook_utils

# Recarregar o módulo notebook_utils
importlib.reload(notebook_utils)

<module 'notebook_utils' from '/Users/patricia/Documents/code/python-code/behavior-detection/src/notebooks/notebook_utils.py'>

In [11]:
import notebook_utils as nb_utils

# Avaliação dos Modelos
class_metrics_results, avg_metrics_results = nb_utils.evaluate_models(trained_models, X_train, y_train, X_test, y_test)

# Geração dos Relatórios
nb_utils.generate_reports(class_metrics_results, avg_metrics_results, filename_prefix="_Skopt_")

# Salvando os modelos em arquivos para recuperação
nb_utils.save_models(trained_models, filename_prefix="_Skopt_")

Model 'Random Forest_none' saved at: ../models/_Skopt__Random Forest_none_20241022_1743.pkl
Model 'XGBoost_none' saved at: ../models/_Skopt__XGBoost_none_20241022_1743.pkl
Modelos salvos: ['../models/_Skopt__Random Forest_none_20241022_1743.pkl', '../models/_Skopt__XGBoost_none_20241022_1743.pkl']


['../models/_Skopt__Random Forest_none_20241022_1743.pkl',
 '../models/_Skopt__XGBoost_none_20241022_1743.pkl']

## Usando Optuna (Otimização Bayesiana)

In [12]:
# Importação da nova classe OptunaBayesianOptimizationTraining
from core.training.optuna_bayesian_optimization_training import OptunaBayesianOptimizationTraining

# Instanciação da classe de treinamento com Otimização Bayesiana via Optuna
training = OptunaBayesianOptimizationTraining()

# Executar o treinamento
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,  # Será mapeado para n_trials na classe OptunaBayesianOptimizationTraining
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Exemplo de acesso aos modelos treinados
for model_key, model_info in trained_models.items():
    print(f"Modelo: {model_key}")
    print(f"Melhores Hiperparâmetros: {model_info['hyperparameters']}")
    print(f"Resultado CV: {model_info['cv_result']}\n")


2024-10-22 17:43:12,636 | optuna_training | INFO | Training and evaluating Random Forest with Optuna Bayesian Optimization and none
INFO:optuna_training:Training and evaluating Random Forest with Optuna Bayesian Optimization and none
[I 2024-10-22 17:43:12,637] A new study created in memory with name: no-name-3b27f138-f203-4d84-9f49-c3f89f7ef34f


Training and evaluating Random Forest with Optuna Bayesian Optimization and none


[I 2024-10-22 17:43:13,121] Trial 0 finished with value: 0.9530043552212628 and parameters: {'classifier__n_estimators': 100, 'classifier__max_depth': 30, 'classifier__min_samples_split': 4, 'classifier__min_samples_leaf': 6}. Best is trial 0 with value: 0.9530043552212628.
[I 2024-10-22 17:43:13,467] Trial 1 finished with value: 0.9362958372324336 and parameters: {'classifier__n_estimators': 87, 'classifier__max_depth': 5, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 3}. Best is trial 0 with value: 0.9530043552212628.
[I 2024-10-22 17:43:13,669] Trial 2 finished with value: 0.9265391057671369 and parameters: {'classifier__n_estimators': 50, 'classifier__max_depth': 5, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 4}. Best is trial 0 with value: 0.9530043552212628.
[I 2024-10-22 17:43:14,055] Trial 3 finished with value: 0.9485438553124871 and parameters: {'classifier__n_estimators': 84, 'classifier__max_depth': 30, 'classifier__min_samples

Training and evaluating XGBoost with Optuna Bayesian Optimization and none


[I 2024-10-22 17:44:06,202] Trial 0 finished with value: 0.9599799071278493 and parameters: {'classifier__n_estimators': 212, 'classifier__learning_rate': 0.12051034822326333, 'classifier__max_depth': 8, 'classifier__subsample': 0.8030584986445675, 'classifier__colsample_bytree': 0.7483189184967313, 'classifier__reg_alpha': 0.4360874723530227, 'classifier__reg_lambda': 0.9550830865767829}. Best is trial 0 with value: 0.9599799071278493.
[I 2024-10-22 17:44:07,605] Trial 1 finished with value: 0.9571687177404643 and parameters: {'classifier__n_estimators': 158, 'classifier__learning_rate': 0.08562155617532057, 'classifier__max_depth': 10, 'classifier__subsample': 0.8297109600198731, 'classifier__colsample_bytree': 0.9656775204688605, 'classifier__reg_alpha': 0.9392673884207936, 'classifier__reg_lambda': 0.19823089701004248}. Best is trial 0 with value: 0.9599799071278493.
[I 2024-10-22 17:44:09,242] Trial 2 finished with value: 0.9613378710030546 and parameters: {'classifier__n_estimato

Modelo: Random Forest_none
Melhores Hiperparâmetros: {'classifier__n_estimators': 102, 'classifier__max_depth': 20, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 1}
Resultado CV: 0.9773433679985404

Modelo: XGBoost_none
Melhores Hiperparâmetros: {'classifier__n_estimators': 166, 'classifier__learning_rate': 0.12616967871848617, 'classifier__max_depth': 9, 'classifier__subsample': 0.7003381340294468, 'classifier__colsample_bytree': 0.7557770346251104, 'classifier__reg_alpha': 0.1663533951672295, 'classifier__reg_lambda': 0.935716339942594}
Resultado CV: 0.9677242968377563



### Avaliação e logging

In [13]:
import notebook_utils as nb_utils

# Avaliação dos Modelos
class_metrics_results, avg_metrics_results = nb_utils.evaluate_models(trained_models, X_train, y_train, X_test, y_test)

# Geração dos Relatórios
nb_utils.generate_reports(class_metrics_results, avg_metrics_results, filename_prefix="_Optuna_")

# Salvando os modelos em arquivos para recuperação
nb_utils.save_models(trained_models, filename_prefix="_Optuna_")

Model 'Random Forest_none' saved at: ../models/_Optuna__Random Forest_none_20241022_1746.pkl
Model 'XGBoost_none' saved at: ../models/_Optuna__XGBoost_none_20241022_1746.pkl
Modelos salvos: ['../models/_Optuna__Random Forest_none_20241022_1746.pkl', '../models/_Optuna__XGBoost_none_20241022_1746.pkl']


['../models/_Optuna__Random Forest_none_20241022_1746.pkl',
 '../models/_Optuna__XGBoost_none_20241022_1746.pkl']

## Using GridSearchCV

In [14]:
from core.training.grid_search_training import GridSearchTraining

# Instantiate the GridSearchCV training class
training = GridSearchTraining()

# Execute the training
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,  # This parameter is not used in GridSearchCV but kept for consistency
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Example of accessing the trained models
for model_key, model_info in trained_models.items():
    print(f"Model: {model_key}")
    print(f"Best Hyperparameters: {model_info['hyperparameters']}")
    print(f"CV Result: {model_info['cv_result']}\n")

2024-10-22 17:46:41,719 | grid_search_training | INFO | Training and evaluating Random Forest with GridSearchCV and none
INFO:grid_search_training:Training and evaluating Random Forest with GridSearchCV and none
DEBUG:grid_search_training:Parameter grid: [{'classifier__n_estimators': [50, 100, 200]}, {'classifier__max_depth': [None, 3, 5, 10, 20, 30]}, {'classifier__min_samples_split': [2, 5, 10]}, {'classifier__min_samples_leaf': [1, 2, 4, 6, 8, 10]}]
2024-10-22 17:46:41,720 | grid_search_training | INFO | Starting GridSearchCV fitting process
INFO:grid_search_training:Starting GridSearchCV fitting process


Fitting 5 folds for each of 18 candidates, totalling 90 fits


2024-10-22 17:46:48,245 | grid_search_training | INFO | GridSearchCV fitting process completed
INFO:grid_search_training:GridSearchCV fitting process completed
2024-10-22 17:46:48,247 | grid_search_training | INFO | Best parameters: {'classifier__max_depth': 30}
INFO:grid_search_training:Best parameters: {'classifier__max_depth': 30}
2024-10-22 17:46:48,247 | grid_search_training | INFO | Best cross-validation score: 0.9753185550082101
INFO:grid_search_training:Best cross-validation score: 0.9753185550082101
2024-10-22 17:46:48,249 | grid_search_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:grid_search_training:All hyperparameter combinations and their cross-validation results:
2024-10-22 17:46:48,250 | grid_search_training | INFO | Params: {'classifier__n_estimators': 50}, Mean Test Score: 0.9724384236453201
INFO:grid_search_training:Params: {'classifier__n_estimators': 50}, Mean Test Score: 0.9724384236453201
2024-10-22 17:46:48,251 | grid

Fitting 5 folds for each of 45 candidates, totalling 225 fits


2024-10-22 17:47:18,022 | grid_search_training | INFO | GridSearchCV fitting process completed
INFO:grid_search_training:GridSearchCV fitting process completed
2024-10-22 17:47:18,023 | grid_search_training | INFO | Best parameters: {'classifier__subsample': 0.7}
INFO:grid_search_training:Best parameters: {'classifier__subsample': 0.7}
2024-10-22 17:47:18,024 | grid_search_training | INFO | Best cross-validation score: 0.9636272342579352
INFO:grid_search_training:Best cross-validation score: 0.9636272342579352
2024-10-22 17:47:18,026 | grid_search_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:grid_search_training:All hyperparameter combinations and their cross-validation results:
2024-10-22 17:47:18,027 | grid_search_training | INFO | Params: {'classifier__n_estimators': 50}, Mean Test Score: 0.9544825290887582
INFO:grid_search_training:Params: {'classifier__n_estimators': 50}, Mean Test Score: 0.9544825290887582
2024-10-22 17:47:18,028 | gr

Model: Random Forest_none
Best Hyperparameters: {'classifier__max_depth': 30}
CV Result: 0.9753185550082101

Model: XGBoost_none
Best Hyperparameters: {'classifier__subsample': 0.7}
CV Result: 0.9636272342579352



### Avaliação e logging

In [15]:
import notebook_utils as nb_utils

# Avaliação dos Modelos
class_metrics_results, avg_metrics_results = nb_utils.evaluate_models(trained_models, X_train, y_train, X_test, y_test)

# Geração dos Relatórios
nb_utils.generate_reports(class_metrics_results, avg_metrics_results, filename_prefix="_GridSearch_")

# Salvando os modelos em arquivos para recuperação
nb_utils.save_models(trained_models, filename_prefix="_GridSearch_")

Model 'Random Forest_none' saved at: ../models/_GridSearch__Random Forest_none_20241022_1747.pkl
Model 'XGBoost_none' saved at: ../models/_GridSearch__XGBoost_none_20241022_1747.pkl
Modelos salvos: ['../models/_GridSearch__Random Forest_none_20241022_1747.pkl', '../models/_GridSearch__XGBoost_none_20241022_1747.pkl']


['../models/_GridSearch__Random Forest_none_20241022_1747.pkl',
 '../models/_GridSearch__XGBoost_none_20241022_1747.pkl']

## Treinando com RandomSearchCV

In [16]:
# src/notebooks/model_training_behavior_multiclassification_by_student_level.ipynb

from core.training.random_search_training import RandomSearchTraining

# Instantiate the RandomizedSearchCV training class
training = RandomSearchTraining()

# Execute the training
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Example of accessing the trained models
for model_key, model_info in trained_models.items():
    print(f"Model: {model_key}")
    print(f"Best Hyperparameters: {model_info['hyperparameters']}")
    print(f"CV Result: {model_info['cv_result']}\n")

2024-10-22 17:47:18,359 | random_search_training | INFO | Training and evaluating Random Forest with RandomizedSearchCV and none
INFO:random_search_training:Training and evaluating Random Forest with RandomizedSearchCV and none
DEBUG:random_search_training:Parameter grid: [{'classifier__n_estimators': [50, 100, 200]}, {'classifier__max_depth': [None, 3, 5, 10, 20, 30]}, {'classifier__min_samples_split': [2, 5, 10]}, {'classifier__min_samples_leaf': [1, 2, 4, 6, 8, 10]}]
2024-10-22 17:47:18,362 | random_search_training | INFO | Starting RandomizedSearchCV fitting process
INFO:random_search_training:Starting RandomizedSearchCV fitting process


Fitting 5 folds for each of 18 candidates, totalling 90 fits


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 18 is smaller than n_iter=100. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
2024-10-22 17:47:24,591 | random_search_training | INFO | RandomizedSearchCV fitting process completed
INFO:random_search_training:RandomizedSearchCV fitting process completed
2024-10-22 17:47:24,592 | random_search_training | INFO | Best parameters: {'classifier__n_estimators': 200}
INFO:random_search_training:Best parameters: {'classifier__n_estimators': 200}
2024-10-22 17:47:24,593 | random_search_training | INFO | Best cross-validation score: 0.9760116767013318
INFO:random_search_training:Best cross-validation score: 0.9760116767013318
2024-10-22 17:47:24,594 | random_search_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:random_search_training:All hyperparameter combinations and their 

Fitting 5 folds for each of 45 candidates, totalling 225 fits


2024-10-22 17:47:52,821 | random_search_training | INFO | RandomizedSearchCV fitting process completed
INFO:random_search_training:RandomizedSearchCV fitting process completed
2024-10-22 17:47:52,822 | random_search_training | INFO | Best parameters: {'classifier__subsample': 0.7}
INFO:random_search_training:Best parameters: {'classifier__subsample': 0.7}
2024-10-22 17:47:52,823 | random_search_training | INFO | Best cross-validation score: 0.9636272342579352
INFO:random_search_training:Best cross-validation score: 0.9636272342579352
2024-10-22 17:47:52,825 | random_search_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:random_search_training:All hyperparameter combinations and their cross-validation results:
2024-10-22 17:47:52,826 | random_search_training | INFO | Params: {'classifier__n_estimators': 50}, Mean Test Score: 0.9544825290887582
INFO:random_search_training:Params: {'classifier__n_estimators': 50}, Mean Test Score: 0.9544825290887

Model: Random Forest_none
Best Hyperparameters: {'classifier__n_estimators': 200}
CV Result: 0.9760116767013318

Model: XGBoost_none
Best Hyperparameters: {'classifier__subsample': 0.7}
CV Result: 0.9636272342579352



### Avaliação e logging

In [17]:
import notebook_utils as nb_utils

# Avaliação dos Modelos
class_metrics_results, avg_metrics_results = nb_utils.evaluate_models(trained_models, X_train, y_train, X_test, y_test)

# Geração dos Relatórios
nb_utils.generate_reports(class_metrics_results, avg_metrics_results, filename_prefix="_RandomSearch_")

# Salvando os modelos em arquivos para recuperação
nb_utils.save_models(trained_models, filename_prefix="_RandomSearch_")

Model 'Random Forest_none' saved at: ../models/_RandomSearch__Random Forest_none_20241022_1747.pkl
Model 'XGBoost_none' saved at: ../models/_RandomSearch__XGBoost_none_20241022_1747.pkl
Modelos salvos: ['../models/_RandomSearch__Random Forest_none_20241022_1747.pkl', '../models/_RandomSearch__XGBoost_none_20241022_1747.pkl']


['../models/_RandomSearch__Random Forest_none_20241022_1747.pkl',
 '../models/_RandomSearch__XGBoost_none_20241022_1747.pkl']